In [20]:
## install packages

import ensurepip
ensurepip.bootstrap()

import sys
!{sys.executable} -m pip install mrcfile
!{sys.executable} -m pip install scipy


Looking in links: /var/folders/kg/k6zd0k716gx82t5t6qmvnvxr0000gq/T/tmpvaaqcosb

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: /programs/i386-mac/jupyterlab/4.4.0/.pixi/envs/default/bin/python -m pip install --upgrade pip

[notice] A new release of pip is available: 25.0.1 -> 25.1
[notice] To update, run: /programs/i386-mac/jupyterlab/4.4.0/.pixi/envs/default/bin/python -m pip install --upgrade pip


In [58]:
import mrcfile
import numpy as np
from scipy.ndimage import gaussian_filter

In [61]:
## Use mrcfile package to create a rectangle mask for mrc_2d_curve_weaken

pixel_size = 0.94 # in Angstrom

# the filament you work on
filament_diameter = 90 # in Angstrom
repeating_unit = 10 # in Angstrom, repeating unit length along filament major axis

# boxsize, you need to calculate by yourself, set a size like 1.5 to 1.6 times pixels larger than the filament diameter 
box_size_x = int (filament_diameter/pixel_size * 1.5)
box_size_y = int (repeating_unit/pixel_size)

if box_size_x % 2 == 0:
    box_size_x = box_size_x
else:
    box_size_x = box_size_x + 1

if box_size_y % 2 == 0:
    box_size_y = box_size_y
else:
    box_size_y = box_size_y + 1

## create a numpu array 
mask = np.zeros((box_size_y, box_size_x), dtype=np.float32)

# Define rectangular mask area, cover the filament
# the mask should be a little bigger than the actuall filament diameter, here is 1.2 times bigger 
filament_diameter_large = filament_diameter*1.2
filament_diameter_large_pixel = filament_diameter_large/pixel_size
x_start = int (box_size_x/2 - filament_diameter_large_pixel/2)
x_end = int (box_size_x/2 + filament_diameter_large_pixel/2 )


# Set rectangular region to 1.0
mask[0:box_size_y, x_start:x_end] = 1.0

# soften edges using Gaussian blur
# adjust sigma for softness, 2 is quite good
mask = gaussian_filter(mask, sigma=2)  

# Save to MRC file
with mrcfile.new('rectangular_mask.mrc', overwrite=True) as mrc:
    mrc.set_data(mask)
    mrc.voxel_size = pixel_size